In [32]:
import numpy as np
import pandas as pd
import re
import collections
import operator
import simpy

ModuleNotFoundError: No module named 'simpy'

In [2]:
trasacao = pd.read_csv("data/POS_transactions_20190328-20190410.csv")

In [3]:
trasacao.head()

,WorkstationGroupID,TranID,BeginDateTime,EndDateTime,OperatorID,TranTime,BreakTime,ArtNum,TNcash,TNcard,Amount
0,1,1.904031e+13,2019-04-03T13:28:17,2019-04-03T13:28:17,10,0,1,1,False,False,11.99
1,1,1.903291e+13,2019-03-29T14:04:50,2019-03-29T14:05:31,101,41,21,6,True,False,13.74
2,1,1.903291e+13,2019-03-29T14:05:52,2019-03-29T14:06:35,101,43,44,12,True,False,97.47
3,1,1.903291e+13,2019-03-29T14:07:19,2019-03-29T14:07:43,101,24,25,6,False,True,37.18
4,1,1.903291e+13,2019-03-29T14:08:08,2019-03-29T14:09:44,101,96,18,48,True,False,154.70


In [4]:
trasacao.dtypes

WorkstationGroupID      int64
TranID                float64
BeginDateTime          object
EndDateTime            object
OperatorID              int64
TranTime                int64
BreakTime               int64
ArtNum                  int64
TNcash                   bool
TNcard                   bool
Amount                float64
dtype: object

In [5]:
trasacao_workstation_service = trasacao.query('WorkstationGroupID == 1')

In [6]:
trasacao_workstation_service_date = trasacao_workstation_service.loc[trasacao_workstation_service.BeginDateTime.str.contains('2019-04-03T10|2019-04-03T11|2019-04-03T12', flags=re.I, regex=True)]

## Estatistica descritiva campo TranTime

In [7]:
print("media",trasacao_workstation_service_date["TranTime"].mean())
print("mediana",trasacao_workstation_service_date["TranTime"].median())
print("Moda",list(trasacao_workstation_service_date["TranTime"].mode()))
print("minimo",trasacao_workstation_service_date["TranTime"].min())
print("maximo",trasacao_workstation_service_date["TranTime"].max())
print("amplitude",trasacao_workstation_service_date["TranTime"].max()-trasacao_workstation_service_date["TranTime"].min())
print("desvio padrão",trasacao_workstation_service_date["TranTime"].std())
print("variacia",trasacao_workstation_service_date["TranTime"].var())
print("coeficinte de variação",trasacao_workstation_service_date["TranTime"].std()/trasacao_workstation_service_date["TranTime"].mean())
print("Assimetria",trasacao_workstation_service_date["TranTime"].skew())

media 50.55750487329435
mediana 41.0
Moda [26, 31]
minimo 0
maximo 309
amplitude 309
desvio padrão 40.12457538731323
variacia 1609.9815500121827
coeficinte de variação 0.7936423185414746
Assimetria 2.0333015820729505


## Estatistica descritiva campo Breaktime

In [8]:
print("media",trasacao_workstation_service_date["BreakTime"].mean())
print("mediana",trasacao_workstation_service_date["BreakTime"].median())
print("Moda",list(trasacao_workstation_service_date["BreakTime"].mode()))
print("minimo",trasacao_workstation_service_date["BreakTime"].min())
print("maximo",trasacao_workstation_service_date["BreakTime"].max())
print("amplitude",trasacao_workstation_service_date["BreakTime"].max()-trasacao_workstation_service_date["TranTime"].min())
print("desvio padrão",trasacao_workstation_service_date["BreakTime"].std())
print("variacia",trasacao_workstation_service_date["BreakTime"].var())
print("coeficinte de variação",trasacao_workstation_service_date["BreakTime"].std()/trasacao_workstation_service_date["TranTime"].mean())
print("Assimetria",trasacao_workstation_service_date["BreakTime"].skew())

media 33.44054580896686
mediana 21.0
Moda [17]
minimo 6
maximo 1147
amplitude 1147
desvio padrão 71.21608892361684
variacia 5071.731321576502
coeficinte de variação 1.4086155774913416
Assimetria 12.022677107628233


## Estatistica Descritiva campo numeros de artigos

In [9]:
print("media",trasacao_workstation_service_date["ArtNum"].mean())
print("mediana",trasacao_workstation_service_date["ArtNum"].median())
print("Moda",list(trasacao_workstation_service_date["ArtNum"].mode()))
print("minimo",trasacao_workstation_service_date["ArtNum"].min())
print("maximo",trasacao_workstation_service_date["ArtNum"].max())
print("amplitude",trasacao_workstation_service_date["ArtNum"].max()-trasacao_workstation_service_date["TranTime"].min())
print("desvio padrão",trasacao_workstation_service_date["ArtNum"].std())
print("variacia",trasacao_workstation_service_date["ArtNum"].var())
print("coeficinte de variação",trasacao_workstation_service_date["ArtNum"].std()/trasacao_workstation_service_date["TranTime"].mean())
print("Assimetria",trasacao_workstation_service_date["ArtNum"].skew())

media 15.19103313840156
mediana 11.0
Moda [3]
minimo 1
maximo 123
amplitude 123
desvio padrão 14.937191337843698
variacia 223.1196850633528
coeficinte de variação 0.2954495356382564
Assimetria 2.6803168909075863


## Estatistica Descritiva campo TNcash e TNcard

In [10]:
trasacao_workstation_service_date["TNcash"].describe()

count      513
unique       2
top       True
freq       335
Name: TNcash, dtype: object

In [30]:
frequencia=collections.Counter(trasacao_workstation_service_date["ArtNum"])

In [29]:
frequencia_ordenada=sorted(frequencia.items(), key=operator.itemgetter(1),reverse=True)

In [31]:
frequencia_ordenada

[(3, 36),
 (4, 34),
 (8, 30),
 (6, 30),
 (7, 26),
 (12, 25),
 (10, 25),
 (9, 22),
 (5, 21),
 (11, 20),
 (13, 19),
 (20, 16),
 (2, 14),
 (15, 14),
 (14, 14),
 (16, 13),
 (1, 13),
 (21, 12),
 (17, 12),
 (19, 10),
 (26, 9),
 (22, 8),
 (24, 7),
 (30, 6),
 (18, 5),
 (27, 5),
 (23, 4),
 (37, 4),
 (42, 4),
 (31, 4),
 (40, 3),
 (25, 3),
 (48, 3),
 (33, 3),
 (41, 2),
 (44, 2),
 (39, 2),
 (35, 2),
 (34, 2),
 (38, 2),
 (28, 2),
 (36, 2),
 (47, 2),
 (62, 2),
 (58, 1),
 (67, 1),
 (59, 1),
 (32, 1),
 (98, 1),
 (82, 1),
 (45, 1),
 (123, 1),
 (53, 1),
 (29, 1),
 (87, 1),
 (49, 1),
 (76, 1),
 (51, 1),
 (46, 1),
 (64, 1),
 (70, 1),
 (72, 1),
 (84, 1)]

In [13]:
t=trasacao_workstation_service_date.head()

## Calculando o tempo medio de item por segundo

In [26]:

produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date.iterrows():
    produtos_segundos += trasacao['ArtNum']/np.add(trasacao["TranTime"],trasacao["BreakTime"])
media_produtos_segundos=len(trasacao_workstation_service_date.index)/produtos_segundos

In [27]:
media_produtos_segundos

5.475793093049854

In [16]:
OperatorIDs=trasacao_workstation_service_date[['OperatorID']]

In [17]:
operadores_unicos = OperatorIDs.drop_duplicates()
operadores_list = operadores_unicos.values

In [18]:
operadores = {}
for operador in operadores_list:
    operadores[int(operador)] = trasacao_workstation_service_date.query(f"OperatorID=={operador}")


In [19]:
operadores.keys()

dict_keys([105, 121, 128, 131, 301, 311, 317, 361])

In [20]:
trasacao_workstation_service_date.describe()

,WorkstationGroupID,TranID,OperatorID,TranTime,BreakTime,ArtNum,Amount
count,513.0,5.130000e+02,513.000000,513.000000,513.000000,513.000000,513.000000
mean,1.0,6.945371e+13,219.707602,50.557505,33.440546,15.191033,62.457505
std,0.0,8.018963e+13,94.280977,40.124575,71.216089,14.937191,78.536479
min,1.0,1.904031e+11,105.000000,0.000000,6.000000,1.000000,2.170000
25%,1.0,1.904031e+13,128.000000,24.000000,15.000000,6.000000,21.450000
50%,1.0,1.904031e+13,301.000000,41.000000,21.000000,11.000000,41.670000
75%,1.0,1.904031e+14,311.000000,67.000000,32.000000,20.000000,74.080000
max,1.0,1.904031e+14,361.000000,309.000000,1147.000000,123.000000,811.750000


In [21]:
trasacao_workstation_service_date_operador301 = trasacao_workstation_service_date.query("OperatorID == 301")


In [22]:
trasacao_workstation_service_date_operador311 = trasacao_workstation_service_date.query("OperatorID == 311")
trasacao_workstation_service_date_operador311.tail()

,WorkstationGroupID,TranID,BeginDateTime,EndDateTime,OperatorID,TranTime,BreakTime,ArtNum,TNcash,TNcard,Amount
44296,1,1.904031e+13,2019-04-03T12:53:40,2019-04-03T12:54:03,311,23,17,7,False,True,11.29
44297,1,1.904031e+13,2019-04-03T12:54:20,2019-04-03T12:55:25,311,65,15,3,True,False,13.42
44298,1,1.904031e+13,2019-04-03T12:55:40,2019-04-03T12:57:11,311,91,10,22,False,True,60.26
44299,1,1.904031e+13,2019-04-03T12:57:21,2019-04-03T12:58:00,311,39,17,12,True,False,63.88
44300,1,1.904031e+13,2019-04-03T12:58:17,2019-04-03T12:59:22,311,65,47,14,True,False,59.82
